In [ ]:
# generate_data.py
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import trange

def significance_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    else:
        return ""

def get_rewards(bids, auction_type="first"):
    """
    Generalized for multiple bidders.
    bids: array of shape (n_bidders,).
    auction_type: "first" or "second".

    Returns:
      (rewards, winner, winner_bid)
        - rewards: array of shape (n_bidders,) with payoff for each bidder
        - winner: integer index of the winning bidder
        - winner_bid: the winning bidder's actual bid

    We assume valuations[i] = 1 for all i (can be changed if desired).
    """
    n_bidders = len(bids)
    valuations = np.ones(n_bidders)  # or some other distribution if desired

    sorted_indices = np.argsort(bids)[::-1]  # descending
    highest_idx = [sorted_indices[0]]
    highest_bid = bids[sorted_indices[0]]
    rewards = np.zeros(n_bidders)

    # Check if there's a tie among the top
    for idx in sorted_indices[1:]:
        if np.isclose(bids[idx], highest_bid):
            highest_idx.append(idx)
        else:
            break

    # Resolve tie by random choice among highest_idx
    if len(highest_idx) > 1:
        winner = np.random.choice(highest_idx)
    else:
        winner = highest_idx[0]
    winner_bid = bids[winner]

    # Second-highest
    if len(highest_idx) == n_bidders:
        second_highest_bid = highest_bid
    else:
        second_idx = None
        for idx in sorted_indices:
            if idx not in highest_idx:
                second_idx = idx
                break
        second_highest_bid = bids[second_idx] if second_idx is not None else highest_bid

    # Payoffs
    if auction_type == "first":
        rewards[winner] = valuations[winner] - winner_bid
    else:  # second-price
        rewards[winner] = valuations[winner] - second_highest_bid

    return rewards, winner, winner_bid


def build_state_space(median_opp_past_bid_index, winner_bid_index_state, n_actions):
    """
    Decide how many total states we have, based on:
      - median_opp_past_bid_index (bool)
      - winner_bid_index_state (bool)
      - n_actions = 6

    If both are False -> 1 state
    If exactly one is True -> n_actions states
    If both True -> n_actions * n_actions states

    We'll define a helper function to map (median_idx, winner_idx) -> single integer in [0..n_states-1].
    """
    if not median_opp_past_bid_index and not winner_bid_index_state:
        n_states = 1
    elif median_opp_past_bid_index and not winner_bid_index_state:
        n_states = n_actions
    elif not median_opp_past_bid_index and winner_bid_index_state:
        n_states = n_actions
    else:
        # both True
        n_states = n_actions * n_actions

    return n_states

def state_to_index(median_idx, winner_idx, median_flag, winner_flag, n_actions):
    """
    Convert the pair (median_idx, winner_idx) into a single integer.

    Cases:
     1) If both flags are False -> always return 0 (only 1 state).
     2) If exactly one flag is True -> return median_idx OR winner_idx as the state ID.
     3) If both are True -> we do a 2D -> 1D mapping: state_id = median_idx * n_actions + winner_idx
    """
    if not median_flag and not winner_flag:
        return 0
    elif median_flag and not winner_flag:
        return median_idx
    elif not median_flag and winner_flag:
        return winner_idx
    else:
        # both True
        return median_idx * n_actions + winner_idx


def run_experiment(alpha, gamma, episodes, auction_type, init, exploration,
                   asynchronous, n_bidders, median_opp_past_bid_index,
                   winner_bid_index_state,
                   seed=0):
    """
    Runs a single experiment with multiple bidders, asynchronous vs synchronous updates,
    optionally using:
      - median_of_opponents' last bids (median_opp_past_bid_index)
      - past winning bid (winner_bid_index_state)

    Returns dict of outcome metrics:
      - avg_rev_last_1000
      - time_to_converge
      - avg_regret_of_seller
    """

    np.random.seed(seed)
    random.seed(seed)

    n_actions = 6
    action_space = np.linspace(0, 1, n_actions)

    # Build state space size
    n_states = build_state_space(median_opp_past_bid_index, winner_bid_index_state, n_actions)

    # Q shape: (n_bidders, n_states, n_actions)
    if init == "random":
        Q = np.random.rand(n_bidders, n_states, n_actions)
    else:
        Q = np.zeros((n_bidders, n_states, n_actions))

    # We'll track the previous round's bids to compute median; start with zeros
    prev_bids = np.zeros(n_bidders)

    # We'll track the previous winning bid (start at 0.0, effectively index=0)
    prev_winner_bid = 0.0
    # Discretize a continuous bid to nearest action idx
    def bid_to_state_index(bid):
        return np.argmin(np.abs(action_space - bid))

    revenues = []
    window_size = 1000

    start_eps, end_eps = 1.0, 0.0
    decay_end = int(0.9 * episodes)

    # Main training loop
    for ep in range(episodes):
        # Epsilon
        if ep < decay_end:
            eps = start_eps - (ep / decay_end) * (start_eps - end_eps)
        else:
            eps = end_eps

        # Determine median index among opponents if needed
        median_idx = 0  # default
        if median_opp_past_bid_index:
            # For simplicity, we compute the median of "other" bidders for each agent
            # but in typical usage, we might do 1 median for each agent's perspective
            # OR a single global median (like we do for "opponents" excluding me).
            # Let's define a "global" median of all bids from last round:
            median_val = np.median(prev_bids)
            median_idx = bid_to_state_index(median_val)

        # Determine previous winner index if needed
        winner_idx = 0
        if winner_bid_index_state:
            winner_idx = bid_to_state_index(prev_winner_bid)

        # State for all bidders is the same if we interpret "opponent median" globally
        # and "previous winner's bid" globally.
        s = state_to_index(
            median_idx,
            winner_idx,
            median_opp_past_bid_index,
            winner_bid_index_state,
            n_actions
        )

        # Each bidder chooses an action
        chosen_actions = []
        for i in range(n_bidders):
            if exploration == "egreedy":
                if np.random.rand() > eps:
                    a_i = np.argmax(Q[i, s])
                else:
                    a_i = np.random.randint(n_actions)
            elif exploration == "boltzmann":
                qvals = Q[i, s]
                exp_q = np.exp(qvals - np.max(qvals))
                probs = exp_q / np.sum(exp_q)
                a_i = np.random.choice(range(n_actions), p=probs)
            else:
                # default egreedy
                if np.random.rand() > eps:
                    a_i = np.argmax(Q[i, s])
                else:
                    a_i = np.random.randint(n_actions)
            chosen_actions.append(a_i)

        # Convert to bids
        bids = np.array([action_space[a_i] for a_i in chosen_actions])

        # Compute rewards, find winner + winning bid
        rewards, winner, winner_bid = get_rewards(bids, auction_type)

        # Q update
        if asynchronous == 1:
            for i in range(n_bidders):
                old_q = Q[i, s, chosen_actions[i]]
                td_target = rewards[i] + gamma * np.max(Q[i, s])
                Q[i, s, chosen_actions[i]] = old_q + alpha * (td_target - old_q)
        else:
            # synchronous
            for i in range(n_bidders):
                cf_rewards = np.zeros(n_actions)
                for a_alt in range(n_actions):
                    cf_bids = bids.copy()
                    cf_bids[i] = action_space[a_alt]
                    cf_r, _, _ = get_rewards(cf_bids, auction_type)
                    cf_rewards[a_alt] = cf_r[i]

                max_next_q = np.max(Q[i, s])
                Q[i, s, :] = (1 - alpha)*Q[i, s, :] + \
                             alpha*(cf_rewards + gamma * max_next_q)

        # Seller revenue
        revenue_t = np.max(bids)
        revenues.append(revenue_t)

        # Update prev_bids
        prev_bids = bids
        # Update prev_winner_bid for next state
        prev_winner_bid = winner_bid

    # Final stats
    if len(revenues) >= window_size:
        avg_rev_last_1000 = np.mean(revenues[-window_size:])
    else:
        avg_rev_last_1000 = np.mean(revenues)

    final_rev = avg_rev_last_1000
    rev_series = pd.Series(revenues)
    roll_avg = rev_series.rolling(window_size).mean()

    lower_band = 0.95 * final_rev
    upper_band = 1.05 * final_rev
    time_to_converge = episodes
    for t in range(len(revenues) - window_size):
        window_val = roll_avg.iloc[t + window_size - 1]
        if (window_val >= lower_band) and (window_val <= upper_band):
            stay_in_band = True
            for j in range(t + window_size, len(revenues) - window_size):
                v_j = roll_avg.iloc[j + window_size - 1]
                if not (lower_band <= v_j <= upper_band):
                    stay_in_band = False
                    break
            if stay_in_band:
                time_to_converge = t + window_size
                break

    regrets = [1.0 - r for r in revenues]
    avg_regret_of_seller = np.mean(regrets)

    return {
        "avg_rev_last_1000": avg_rev_last_1000,
        "time_to_converge": time_to_converge,
        "avg_regret_of_seller": avg_regret_of_seller
    }


def run_experiment_with_revenues(alpha, gamma, episodes, auction_type, init, 
                                 exploration, asynchronous, n_bidders,
                                 median_opp_past_bid_index,
                                 winner_bid_index_state,
                                 seed=0):
    """
    Same logic, but returns entire revenue sequence (useful for plotting).
    """

    np.random.seed(seed)
    random.seed(seed)

    n_actions = 6
    action_space = np.linspace(0, 1, n_actions)

    n_states = build_state_space(median_opp_past_bid_index, winner_bid_index_state, n_actions)

    if init == "random":
        Q = np.random.rand(n_bidders, n_states, n_actions)
    else:
        Q = np.zeros((n_bidders, n_states, n_actions))

    prev_bids = np.zeros(n_bidders)
    prev_winner_bid = 0.0

    def bid_to_state_index(bid):
        return np.argmin(np.abs(action_space - bid))

    revenues = []

    start_eps, end_eps = 1.0, 0.0
    decay_end = int(0.9 * episodes)

    for ep in range(episodes):
        if ep < decay_end:
            eps = start_eps - (ep / decay_end) * (start_eps - end_eps)
        else:
            eps = end_eps

        # Build the state index
        median_idx = 0
        if median_opp_past_bid_index:
            median_val = np.median(prev_bids)
            median_idx = bid_to_state_index(median_val)

        winner_idx = 0
        if winner_bid_index_state:
            winner_idx = bid_to_state_index(prev_winner_bid)

        # Single state s
        s = state_to_index(
            median_idx, winner_idx,
            median_opp_past_bid_index, winner_bid_index_state,
            n_actions
        )

        chosen_actions = []
        for i in range(n_bidders):
            if exploration == "egreedy":
                if np.random.rand() > eps:
                    a_i = np.argmax(Q[i, s])
                else:
                    a_i = np.random.randint(n_actions)
            else:
                # default egreedy
                if np.random.rand() > eps:
                    a_i = np.argmax(Q[i, s])
                else:
                    a_i = np.random.randint(n_actions)
            chosen_actions.append(a_i)

        bids = np.array([action_space[a_i] for a_i in chosen_actions])
        rewards, winner, winner_bid = get_rewards(bids, auction_type)

        if asynchronous == 1:
            for i in range(n_bidders):
                old_q = Q[i, s, chosen_actions[i]]
                td_target = rewards[i] + gamma * np.max(Q[i, s])
                Q[i, s, chosen_actions[i]] = old_q + alpha * (td_target - old_q)
        else:
            for i in range(n_bidders):
                cf_rewards = np.zeros(n_actions)
                for a_alt in range(n_actions):
                    cf_bids = bids.copy()
                    cf_bids[i] = action_space[a_alt]
                    cf_r, _, _ = get_rewards(cf_bids, auction_type)
                    cf_rewards[a_alt] = cf_r[i]

                max_next_q = np.max(Q[i, s])
                Q[i, s, :] = (1 - alpha)*Q[i, s, :] + \
                             alpha*(cf_rewards + gamma * max_next_q)

        revenues.append(np.max(bids))

        prev_bids = bids
        prev_winner_bid = winner_bid

    return revenues


if __name__ == "__main__":
    os.makedirs("experiment1", exist_ok=True)

    # ------------------------------------------------------------------------
    # A) Demonstration with "both" state dimensions turned on
    # ------------------------------------------------------------------------
    alpha_demo = 0.01
    gamma_demo = 0.9
    episodes_demo = 50_000
    init_demo = "random"
    exploration_demo = "egreedy"
    asynchronous_demo = 1
    n_bidders_demo = 2
    median_opp_past_bid_index_demo = True     # includes median-of-opponents
    winner_bid_index_state_demo = True        # includes past winning bid
    seed_demo = 999

    # first-price
    revenues_first = run_experiment_with_revenues(
        alpha=alpha_demo,
        gamma=gamma_demo,
        episodes=episodes_demo,
        auction_type="first",
        init=init_demo,
        exploration=exploration_demo,
        asynchronous=asynchronous_demo,
        n_bidders=n_bidders_demo,
        median_opp_past_bid_index=median_opp_past_bid_index_demo,
        winner_bid_index_state=winner_bid_index_state_demo,
        seed=seed_demo
    )

    # second-price
    revenues_second = run_experiment_with_revenues(
        alpha=alpha_demo,
        gamma=gamma_demo,
        episodes=episodes_demo,
        auction_type="second",
        init=init_demo,
        exploration=exploration_demo,
        asynchronous=asynchronous_demo,
        n_bidders=n_bidders_demo,
        median_opp_past_bid_index=median_opp_past_bid_index_demo,
        winner_bid_index_state=winner_bid_index_state_demo,
        seed=seed_demo
    )

    # Rolling average plot
    window_size = 500
    roll_first = pd.Series(revenues_first).rolling(window_size, min_periods=1).mean()
    roll_second = pd.Series(revenues_second).rolling(window_size, min_periods=1).mean()

    plt.figure(figsize=(8,5))
    plt.plot(roll_first, label="First-price Auction", alpha=0.9)
    plt.plot(roll_second, label="Second-price Auction", alpha=0.9)
    plt.title("Rolling Avg of Revenue: (2 Bidders, Both State Dimensions ON)")
    plt.xlabel("Episode")
    plt.ylabel("Revenue")
    plt.legend()
    plot_path = "experiment1/demo_revenue_comparison.png"
    plt.savefig(plot_path, bbox_inches='tight')
    plt.close()
    print(f"Saved demo revenue comparison plot to {plot_path}")

    # ------------------------------------------------------------------------
    # B) Single progress bar over K experiments
    # ------------------------------------------------------------------------
    param_space = {
        "auction_type": ["first", "second"],
        "init": ["random", "zeros"],
        "exploration": ["egreedy", "boltzmann"],
        "asynchronous": [0, 1],
        "n_bidders": [2, 4, 6],
        "median_opp_past_bid_index": [False, True],
        "winner_bid_index_state": [False, True]
    }

    outcomes = ["avg_rev_last_1000", "time_to_converge", "avg_regret_of_seller"]
    K = 500
    results = []

    for k in trange(K, desc="Overall Experiments"):
        alpha = random.uniform(0.001, 0.1)
        gamma = random.uniform(0.0, 0.99)
        episodes = int(random.uniform(40_000, 60_000))

        # pick from discrete
        auction_type = random.choice(param_space["auction_type"])
        init = random.choice(param_space["init"])
        exploration = random.choice(param_space["exploration"])
        asynchronous = random.choice(param_space["asynchronous"])
        n_bidders = random.choice(param_space["n_bidders"])
        median_flag = random.choice(param_space["median_opp_past_bid_index"])
        winner_flag = random.choice(param_space["winner_bid_index_state"])

        outcome = run_experiment(
            alpha=alpha,
            gamma=gamma,
            episodes=episodes,
            auction_type=auction_type,
            init=init,
            exploration=exploration,
            asynchronous=asynchronous,
            n_bidders=n_bidders,
            median_opp_past_bid_index=median_flag,
            winner_bid_index_state=winner_flag,
            seed=k
        )

        outcome["alpha"] = alpha
        outcome["gamma"] = gamma
        outcome["episodes"] = episodes
        outcome["auction_type"] = auction_type
        outcome["init"] = init
        outcome["exploration"] = exploration
        outcome["asynchronous"] = asynchronous
        outcome["n_bidders"] = n_bidders
        outcome["median_opp_past_bid_index"] = median_flag
        outcome["winner_bid_index_state"] = winner_flag

        results.append(outcome)

    df = pd.DataFrame(results)
    csv_path = "experiment1/data.csv"
    df.to_csv(csv_path, index=False)
    print(f"Data generation complete. Saved to '{csv_path}'")


Saved demo revenue comparison plot to experiment1/demo_revenue_comparison.png


Overall Experiments:  13%|█▎        | 63/500 [27:41<12:32:02, 103.25s/it]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# DoubleML
import doubleml as dml
from doubleml import DoubleMLData, DoubleMLIRM

# LightGBM
from lightgbm import LGBMRegressor, LGBMClassifier

# For spline expansions
import patsy

# ------------------------------------------------------------------------------
# 1) Load your RL experiment data
# ------------------------------------------------------------------------------
df = pd.read_csv("experiment1/data.csv")

# ------------------------------------------------------------------------------
# 2) Preprocess data
# ------------------------------------------------------------------------------
# Treatment T = 'auction_type' -> map "first" -> 0, "second" -> 1
df['D'] = (df['auction_type'] == 'second').astype(int)

# Encode non-numeric categorical columns
categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
for col in categorical_cols:
    df[col] = pd.factorize(df[col])[0]  # Encode categories as integers

# Dynamically detect binary and continuous covariates
binary_cols = [col for col in df.columns if df[col].nunique() == 2 and col not in ['auction_type', 'D']]
cont_cols = [col for col in df.columns if col not in binary_cols + ['auction_type', 'D', 'avg_rev_last_1000', 'time_to_converge', 'avg_regret_of_seller']]
X_cols = binary_cols + cont_cols

print(f"Binary covariates: {binary_cols}")
print(f"Continuous covariates: {cont_cols}")

# ------------------------------------------------------------------------------
# 3) Outcomes
# ------------------------------------------------------------------------------
outcomes = ['avg_rev_last_1000', 'time_to_converge', 'avg_regret_of_seller']

# ------------------------------------------------------------------------------
# 4) Loop over outcomes and perform DoubleML inference
# ------------------------------------------------------------------------------
for outcome in outcomes:
    print(f"\n========== Running for Outcome: {outcome} ==========")

    # Define Y (outcome)
    df['Y'] = df[outcome]

    # Create DoubleMLData object
    dml_data = DoubleMLData(df, y_col='Y', d_cols='D', x_cols=X_cols)

    # Define learners
    ml_g = LGBMRegressor(verbose=-1, random_state=123)
    ml_m = LGBMClassifier(verbose=-1, random_state=123)

    # Define and fit DoubleML IRM
    dml_irm = DoubleMLIRM(dml_data, ml_g=ml_g, ml_m=ml_m, n_folds=3, score='ATE')
    dml_irm.fit()

    # Print ATE results
    print(dml_irm.summary)

    # ------------------------------------------------------------------------------
    # 5) GATE Analysis
    # ------------------------------------------------------------------------------
    n_bin = len(binary_cols)
    nrows_gate = int(np.ceil(n_bin / 3))
    ncols_gate = min(n_bin, 3)

    fig_gate, axes_gate = plt.subplots(nrows=nrows_gate, ncols=ncols_gate, figsize=(5 * ncols_gate, 4 * nrows_gate))

    if n_bin == 1:
        axes_gate = np.array([axes_gate])

    for i, bin_col in enumerate(binary_cols):
        groups_df = df[[bin_col]].astype('category')
        gate_obj = dml_irm.gate(groups=groups_df)
        ci_95_gate = gate_obj.confint(level=0.95)

        effects = ci_95_gate['effect']
        lower_95 = ci_95_gate['2.5 %']
        upper_95 = ci_95_gate['97.5 %']

        ax = axes_gate.flatten()[i] if n_bin > 1 else axes_gate[0]

        x_positions = [0, 1]
        ax.errorbar(
            x_positions, effects,
            yerr=[effects - lower_95, upper_95 - effects],
            fmt='o', capsize=5
        )
        ax.set_title(f"GATE: {bin_col} ({outcome})")
        ax.set_xticks(x_positions)
        ax.set_xticklabels([f"{bin_col}=0", f"{bin_col}=1"])
        ax.set_ylabel("Estimated GATE")

    fig_gate.tight_layout()
    gate_plot_path = os.path.join("experiment1", f"gate_plots_{outcome}.png")
    fig_gate.savefig(gate_plot_path, bbox_inches='tight')
    plt.close(fig_gate)
    print(f"GATE plots saved for {outcome}.")

    # ------------------------------------------------------------------------------
    # 6) CATE Analysis
    # ------------------------------------------------------------------------------
    n_cont = len(cont_cols)
    nrows_cate = int(np.ceil(n_cont / 3))
    ncols_cate = min(n_cont, 3)

    fig_cate, axes_cate = plt.subplots(nrows=nrows_cate, ncols=ncols_cate, figsize=(5 * ncols_cate, 4 * nrows_cate))

    if n_cont == 1:
        axes_cate = np.array([axes_cate])

    for i, cont_col in enumerate(cont_cols):
        design_matrix = patsy.dmatrix(f"bs({cont_col}, df=5, degree=2)", df)
        spline_basis = pd.DataFrame(design_matrix)

        cate_obj = dml_irm.cate(basis=spline_basis)
        ci_95_cate = cate_obj.confint(basis=spline_basis, level=0.95)

        effects_cate = ci_95_cate['effect'].values
        lower_95_cate = ci_95_cate['2.5 %'].values
        upper_95_cate = ci_95_cate['97.5 %'].values

        x_values = df[cont_col].values
        idx_sort = np.argsort(x_values)

        x_sorted = x_values[idx_sort]
        eff_sorted = effects_cate[idx_sort]
        low_sorted = lower_95_cate[idx_sort]
        up_sorted = upper_95_cate[idx_sort]

        ax_cate = axes_cate.flatten()[i] if n_cont > 1 else axes_cate[0]

        ax_cate.plot(x_sorted, eff_sorted, label='CATE')
        ax_cate.fill_between(x_sorted, low_sorted, up_sorted, alpha=0.2, label='95% CI')
        ax_cate.set_title(f"CATE: {cont_col} ({outcome})")
        ax_cate.set_xlabel(cont_col)
        ax_cate.set_ylabel("Estimated Treatment Effect")
        ax_cate.legend()

    fig_cate.tight_layout()
    cate_plot_path = os.path.join("experiment1", f"cate_plots_{outcome}.png")
    fig_cate.savefig(cate_plot_path, bbox_inches='tight')
    plt.close(fig_cate)
    print(f"CATE plots saved for {outcome}.")


Binary covariates: ['init', 'exploration', 'asynchronous', 'n_bidders', 'median_opp_past_bid_index', 'winner_bid_index_state']
Continuous covariates: ['alpha', 'gamma', 'episodes']

========== Running for Outcome: avg_rev_last_1000 ==========
       coef   std err         t     P>|t|     2.5 %    97.5 %
D  0.113469  0.041224  2.752511  0.005914  0.032672  0.194267
GATE plots saved for avg_rev_last_1000.
CATE plots saved for avg_rev_last_1000.

========== Running for Outcome: time_to_converge ==========
           coef      std err         t     P>|t|         2.5 %       97.5 %
D -11045.965083  6327.731023 -1.745644  0.080873 -23448.089991  1356.159825
GATE plots saved for time_to_converge.
CATE plots saved for time_to_converge.

========== Running for Outcome: avg_regret_of_seller ==========
       coef   std err         t     P>|t|     2.5 %    97.5 %
D -0.091252  0.035809 -2.548263  0.010826 -0.161437 -0.021067
GATE plots saved for avg_regret_of_seller.
CATE plots saved for avg_regre